In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from pbo.environment.car_on_hill import CarOnHillEnv
import numpy as np

max_pos = 1.0
max_velocity = 3.0
gamma = 0.9


env = CarOnHillEnv(max_pos, max_velocity, gamma)

In [2]:
import jax.numpy as jnp 


env.reset()
absorbing = False
step = 0


while not absorbing and step < 15:
    env.render()

    _, reward, absorbing, _ = env.step(jnp.array([1]))
    step += 1

env.close()

## Time analysis

In [3]:
from scipy.integrate import odeint


def old_step(environment, action: jnp.ndarray) -> tuple:
    action = environment.actions_values[action[0]]
    sa = np.append(environment.state, action)
    new_state = odeint(environment._dpds, sa, [0, environment._dt])

    environment.state = jnp.array(new_state[-1, :-1])

    if environment.state[0] < -environment.max_pos or np.abs(environment.state[1]) > environment.max_velocity:
        reward = -1.0
        absorbing = True
    elif environment.state[0] > environment.max_pos and np.abs(environment.state[1]) <= environment.max_velocity:
        reward = 1.0
        absorbing = True
    else:
        reward = 0.0
        absorbing = False

    return environment.state, jnp.array([reward]), jnp.array([absorbing]), {}

In [4]:
import time
from scipy.integrate import odeint

env.reset()

t0 = time.time()

for i in range(100):
    old_step(env, jnp.array([i % 2]))

print("5000 steps:", 50 * (time.time() - t0) / 60)

5000 steps: 0.3800457715988159


In [5]:
env.reset()

t0 = time.time()

for i in range(100):
    env.step(jnp.array([i % 2]))

print("5000 steps:", 50 * (time.time() - t0) / 60)

5000 steps: 0.029874642690022785
